In [ ]:
import pandas as pd
import re
import csv

df = pd.read_csv(
    "../data/interim/data_cleaning.csv", low_memory=False, dtype={"ID_orateur": str}
)
df.shape

(215127, 59)

In [ ]:
# nettoyage moche du texte


def nettoyer_texte(texte):
    if not isinstance(texte, str):
        return texte
    # Supprimer les balises HTML/XML
    texte = re.sub(r"<[^>]+>", "", texte)
    # Supprimer contenu entre parenthèses ou crochets
    texte = re.sub(r"\([^)]*\)", "", texte)
    # # TODO: aviser si enlève crochets, ou juste […] [...], surtout des troncatures citations
    # texte = re.sub(r"\[[^\]]*\]", "", texte)
    # Supprimer les espaces multiples
    texte = re.sub(r"\s+", " ", texte).strip()
    return texte


df["Texte_clean"] = df["Texte"].apply(nettoyer_texte)

In [ ]:
# A GARDER POUR LA TRACE DU LOL

# pays_avec_republique = [
#     "République fédérative du Brésil",
#     "République argentine",
#     "République du Bénin",
#     "République de Bolivie",
#     "République du Botswana",
#     "République du Burkina Faso",
#     "République du Burundi",
#     "République du Cameroun",
#     "République du Cap-Vert",
#     "République centrafricaine",
#     "République du Tchad",
#     "République du Chili",
#     "République populaire de Chine",
#     "République de Colombie",
#     "République du Congo",
#     "République démocratique du Congo",
#     "République de Corée",
#     "République populaire démocratique de Corée",
#     "République de Côte d’Ivoire",
#     "République de Croatie",
#     "République de Cuba",
#     "République tchèque",
#     "République du Danemark",
#     "République dominicaine",
#     "République d’Équateur",
#     "République arabe d’Égypte",
#     "République d’El Salvador",
#     "République d’Estonie",
#     "République de Fidji",
#     "République de Finlande",
#     "République gabonaise",
#     "République de Gambie",
#     "République du Ghana",
#     "République de Grèce",
#     "République du Guatemala",
#     "République de Guinée",
#     "République de Guinée-Bissau",
#     "République coopérative du Guyana",
#     "République d’Haïti",
#     "République de Hongrie",
#     "République d’Inde",
#     "République d’Indonésie",
#     "République islamique d’Iran",
#     "République d’Irak",
#     "République d’Irlande",
#     "République italienne",
#     "République du Kazakhstan",
#     "République du Kenya",
#     "République kirghize",
#     "République démocratique populaire lao",
#     "République de Lettonie",
#     "République du Liban",
#     "République du Lesotho",
#     "République du Libéria",
#     "République de Lituanie",
#     "République de Madagascar",
#     "République du Malawi",
#     "République des Maldives",
#     "République du Mali",
#     "République de Malte",
#     "République des Îles Marshall",
#     "République islamique de Mauritanie",
#     "République de Maurice",
#     "République du Mozambique",
#     "République de Namibie",
#     "République du Niger",
#     "République fédérale du Nigeria",
#     "République de Macédoine du Nord",
#     "République de Palau",
#     "République du Panama",
#     "République du Paraguay",
#     "République du Pérou",
#     "République des Philippines",
#     "République de Pologne",
#     "République portugaise",
#     "République du Rwanda",
#     "République de Saint-Marin",
#     "République de Sierra Leone",
#     "République de Singapour",
#     "République slovaque",
#     "République de Slovénie",
#     "République des Seychelles",
#     "République somalienne",
#     "République d’Afrique du Sud",
#     "République du Soudan",
#     "République du Soudan du Sud",
#     "République arabe syrienne",
#     "République de Tanzanie",
#     "République togolaise",
#     "République de Trinité-et-Tobago",
#     "République tunisienne",
#     "République de Turquie",
#     "République du Turkménistan",
#     "République d’Ouganda",
#     "République d’Ukraine",
#     "République orientale de l’Uruguay",
#     "République d’Ouzbékistan",
#     "République bolivarienne du Venezuela",
#     "République socialiste du Viêt Nam",
#     "République du Yémen",
#     "République de Zambie",
#     "République du Zimbabwe"
# ]

# pattern_pays = r"\b(" + r"|".join(re.escape(p) for p in pays_avec_republique) + r")\b"

In [ ]:
# Regex du champ lexical République (simplifié ici)


pattern_lexical = re.compile(
    r"républi",
    re.I,
)

# Regex des expressions à exclure
# ici garder la casse
pattern_excl = re.compile(
    r"\b[Ll]es Républicains\b"
    r"|\b[Pp]résident de la République\b"
    r"|\b[Pp]résidence de la République\b"
    r"\b[Gg]auche démocrate et républicaine\b"
    r"|\b[Rr]épublique en marche\b"
    r"|\b[Pp]rocureur de la République\b"
    r"|\b[Cc]our(?:s)? de justice de la République\b"
    r"|\b[Aa]dministration générale de la République\b"
    # r"|" + pattern_pays  # ajout des exclusions de pays si existe
)


def contains_lexical_outside_excl(text):
    # Trouve toutes les positions des expressions exclues
    excl_positions = [m.span() for m in pattern_excl.finditer(text)]

    # Fonction pour vérifier si une position est dans une zone exclue
    def in_excl(pos):
        for start, end in excl_positions:
            if start <= pos < end:
                return True
        return False

    # Chercher toutes les occurences du champ lexical
    for match in pattern_lexical.finditer(text):
        start_pos = match.start()
        if not in_excl(start_pos):
            return True
    return False


# Appliquer sur la colonne
df["repu_match_valide"] = df["Texte_clean"].apply(contains_lexical_outside_excl)

In [ ]:
df_match = df[df["repu_match_valide"]]
df_match.shape

(4185, 61)

In [ ]:
mon_texte = "la République du Congo y a pas que des bonnes BD"
contains_lexical_outside_excl(mon_texte)

False

In [ ]:
congo = df[df["Texte_clean"].str.contains("Congo")]

In [ ]:
congo